# **ZAugNet Colab Notebook**

This Colab Notebook is related to the paper [Self-Supervised Z-Slice Augmentation for 3D Bio-Imaging via Knowledge Distillation](https://arxiv.org/abs/2503.04843).

It provides an intuitive interface for training and/or testing ZAugNet and ZAugNet+, eliminating the need to interact with the code directly.



### **0 - Data preparation**

* In the Colab settings, check "Runtime > Change runtime type"" and select "T4GPU".

* Run the following blocks to connect your Google Drive and create a new folder where to download ZAugNet:

In [1]:
import os
import shutil
from glob import glob
os.chdir('/content/')

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

### **1 - Download ZAugNet and install the requirements**


* Download ZAugNet from GitHub:

In [ ]:
!git clone https://github.com/VirtualEmbryo/ZAugNet.git

* **(Optional)** Download train/test data images and pre-trained models:

In [ ]:
# Download data and pre-trained models from Zenodo
!curl "https://zenodo.org/records/14961732/files/zenodo.zip?download=1" --output /content/ZAugNet/zaugnet_data_and_models.zip

# Unzip the folder
import shutil
shutil.unpack_archive("/content/ZAugNet/zaugnet_data_and_models.zip", "/content/ZAugNet/zaugnet_data_and_models")

# Create data folder and move files
!mkdir /content/ZAugNet/data
!mv /content/ZAugNet/zaugnet_data_and_models/zenodo/ascidians /content/ZAugNet/data/ascidians
!mv /content/ZAugNet/zaugnet_data_and_models/zenodo/filaments /content/ZAugNet/data/filaments
!mv /content/ZAugNet/zaugnet_data_and_models/zenodo/humans /content/ZAugNet/data/humans
!mv /content/ZAugNet/zaugnet_data_and_models/zenodo/nuclei /content/ZAugNet/data/nuclei
!mv /content/ZAugNet/zaugnet_data_and_models/zenodo/results /content/ZAugNet/results

* Install the requirements:

In [ ]:
os.chdir('./ZAugNet')
%pip install -r requirements.txt

* Insert paths to train and test folders.

You can upload your <code>.tif</code> images on Google Drive and use the path <code>./gdrive/MyDrive/path_folder</code>.

Or you can use our data and use the path <code>/content/ZAugNet/data/ascidiands</code> for example.

In [ ]:
os.chdir('/content')
# Put the adresse of the tif files for training and test data in google drive
train_path = "<path_train_folder>" #@param {type:"string"}
test_path = "<path_test_folder>" #@param {type:"string"}

# Change the directory and create necessary folders
if not os.path.exists('./ZAugNet/data/train') :
  shutil.copytree(train_path, './ZAugNet/data/train/')
if not os.path.exists('./ZAugNet/data/test') :
  shutil.copytree(test_path, './ZAugNet/data/test/')

os.chdir('./ZAugNet')

### **2 - Change ZAugNet settings**

* If needed, you can use the following block to adjust the hyper-parameters:

In [ ]:
from train import train
from config import config

cfg = config()

# Parameters on the model that you want to train or inference
cfg.model_name = 'zaugnet' #@param ["zaugnet", "zaugnet+"]

# If you want to use our pre-trained models, select one of the following options: "ascidians", "nuclei", "filaments", or "humans".
#If you prefer to train your own custom model, select "other".
cfg.dataset = "ascidians" #@param ["ascidians", "nuclei", "filaments", "humans", "other"]

#@markdown * #### *Change these parameters **only if** you want to **train** a new model:*
# Number of total epochs to run
cfg.n_epochs = 10  #@param {type:"number"}
# Mini-batch size for training
cfg.batch_size = 32  #@param {type:"number"}
# Learning rate
cfg.lr = 1e-4  #@param {type:"number"}
# Beta1 for Adam optimizer
cfg.beta1 = 0.5 #@param {type:"number"}
# Beta2 for Adam optimizer
cfg.beta1 = 0.999 #@param {type:"number"}
# Validation split proportion
cfg.p_val = 0.1 #@param {type:"number"}
# Number of discriminator updates per generator update
cfg.n_critic = 5 #@param {type:"number"}
# Weight for adversarial loss
cfg.lambda_adv = 0.001 #@param {type:"number"}
# Weight for gradient penalty
cfg.lambda_gp = 10 #@param {type:"number"}
# Maximum distance for triplet sampling (Only ZAugNet+)
cfg.distance_triplets = 7 #@param {type:"number"}

# @markdown * #### *Change these parameters for **training and inference**:*
# Normalization method for input data
cfg.normalization = 'min_max' #@param ["min_max", "z_score"]
# Resizing strategy for input images
cfg.resize_par = 'resize' #@param ["resize", "crop"]
# Size of input images
cfg.patch_size = 256 #@param {type:"number"}
# Image number of bits: 8 bits or 16 bits
cfg.image_bits = 8 #@param [8, 16]
# Range for min-max normalization (if you data are 16bits put (2**16-1)*1.0)
cfg.min_max = (0, (2**8 - 1)*1.0) if cfg.image_bits == 8 else  (0, (2**16 - 1)*1.0)
# List of GPU device IDs to use for training ("0" for Colab)
device_ids = "0" #@param {type:"string"}
cfg.device_ids = [int(id.strip()) for id in device_ids.split(",")]

### **3 - Training**


* To train a new ZAugNet or ZAugNet+ model, execute the following block.

  The trained model will be saved in <code>/content/ZAugNet/results/\<dataset_name\>/</code> as `.pt` file.



**N.B.** Training a new model may take some time.

To prevent the Colab notebook from disconnecting, ensure you run the next two blocks together.

In [ ]:
train(cfg)

In [ ]:
while True:pass

### **4 - Prediction**

* To predict using a ZAugNet or ZAugNet+ model in the middle execute the following block.

  The predicted images will be saved in <code>/content/ZAugNet/predictions/</code> as a `.tif` files.

In [ ]:
# Import the appropriate prediction module based on the selected model name
from predict import predict
predict(cfg)

**N.B.** You can use ZAugNet+ to predict multiple z-slices at once between two consecutive original z-slices $I_0$ and $I_1$.

Simply specify the desired final resolution below and execute the following block.

<code>factor=2</code> means adding 2 equally spaced z-slices between $I_0$ and $I_1$.

In [ ]:
if cfg.model_name == 'zaugnet+':
  from predict import predict
  # Multiply the number of z-slices by this factor
  factor = 2  #@param {type:"number"}
  cfg.factor = factor
  # Call the prediction function with the options and the scaling factor
  predict(cfg)
else :
  raise ValueError("You have to choose zaugnet+ for this block")

You can use ZAugNet+ to predict at a specefic distance between two consecutive original z-slices $I_0$ and $I_1$ using <code>DPM</code>

In [ ]:
if cfg.model_name == 'zaugnet+':
  from predict import predict
  # Multiply the number of z-slices by this factor
  DPM = 0.5  #@param {type:"number"}
  cfg.DPM = DPM
  # Call the prediction function with the options and the DPM
  predict(cfg)
else :
  raise ValueError("You have to choose zaugnet+ for this block")